In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import os

sns.set(style='ticks')

from LAMMPSdataTK import *
from datetime import datetime

In [2]:
! ls MATSQ_C/

carbon_bed_DREIDING_QeD_.data


In [3]:
MATSQ_filename_ = "MATSQ_C/carbon_bed_DREIDING_QeD_.data"
VMD_filename_   = "carbon_bed_OPLS-AA_QeD_charges.data"

In [4]:
#! subl carbon_bed_OPLS-AA_QeD_charges.data

In [5]:
MATSQ_structure = get_structure(MATSQ_filename_,[36-11,2736-11])
MATSQ_bonds     = get_bonds(MATSQ_filename_,[2739-11,6624-11])
MATSQ_angles    = get_angles(MATSQ_filename_,[6627-11,14077-11])
MATSQ_dihedrals = get_dihedrals(MATSQ_filename_,[14081-12,28435-11])

MATSQ_structure["drediding"] = MATSQ_structure["atom type"].map({1:"C_1",2:"C_R",3:"C_3"})

VMD_structure   = get_structure(VMD_filename_,[36,2736])
VMD_bonds       = get_bonds(VMD_filename_,[2739,6624])
VMD_angles      = get_angles(VMD_filename_,[6627,14077])
VMD_dihedrals   = get_dihedrals(VMD_filename_,[14081,28435])
VMD_impropers   = get_impropers(VMD_filename_,[28438,30818])

### Luckily, `atom id` from MATSQ matches the one from VMD.

In [6]:
dreiding = dict(zip(MATSQ_structure["atom id"].values,MATSQ_structure["drediding"].values))
VMD_structure["drediding"] = VMD_structure["atom id"].map(dreiding)

In [7]:
VMD_bonds["X-X"] = VMD_bonds["atom1"].map(dreiding) + "-" +  VMD_bonds["atom2"].map(dreiding) 

In [8]:
VMD_bonds["dreiding bond type"] = VMD_bonds["X-X"].map({"C_1-C_1":12,"C_1-C_R":13,
                                                        "C_R-C_R":14, "C_R-C_3":15,
                                                        "C_R-C_1":13})

In [9]:
VMD_angles["X-X-X"] = "X-"+VMD_angles["atom2"].map(dreiding)+"-X"

In [10]:
"""
DREIDING
  1     50.000000    180.000000    # X C_1 X (type 14)
  2     50.000000    120.000000    # X C_R X (type 15)
"""
VMD_angles["dreiding angle type"] = VMD_angles["X-X-X"].map({"X-C_1-X":14, "X-C_R-X":15})

In [11]:
VMD_dihedrals["X-X-X-X"] = VMD_dihedrals["atom1"].map(dreiding) + "-" + \
                           VMD_dihedrals["atom2"].map(dreiding) + "-" + \
                           VMD_dihedrals["atom3"].map(dreiding) + "-" + \
                           VMD_dihedrals["atom4"].map(dreiding)

In [12]:
VMD_dihedrals["X-X-X-X"].unique()

array(['C_1-C_1-C_1-C_R', 'C_1-C_1-C_R-C_1', 'C_1-C_1-C_R-C_R',
       'C_1-C_R-C_R-C_1', 'C_1-C_R-C_R-C_R', 'C_R-C_R-C_R-C_1',
       'C_R-C_R-C_R-C_R', 'C_R-C_1-C_1-C_R', 'C_1-C_R-C_1-C_R',
       'C_R-C_R-C_1-C_R', 'C_R-C_1-C_R-C_R', 'C_R-C_1-C_R-C_1',
       'C_R-C_1-C_1-C_1', 'C_R-C_1-C_R-C_3', 'C_3-C_R-C_R-C_R',
       'C_3-C_R-C_R-C_1', 'C_R-C_R-C_1-C_1', 'C_R-C_R-C_R-C_3',
       'C_1-C_R-C_R-C_3'], dtype=object)

In [13]:
"""
DREIDING PARAMETRIZATION

  1      3.125000   -1   2   # X C_1 C_1 X
  2      3.125000   -1   2   # C_R C_1 C_R X
  3      3.125000   -1   2   # X C_1 C_R X
  4      3.125000   -1   2   # X C_R C_R X
  5      3.125000   -1   2   # X C_R C_R X
  6      3.125000   -1   2   # X C_R C_R X
  7      3.125000   -1   2   # X C_R C_R X
  8      3.125000   -1   2   # X C_1 C_1 X
  9      3.125000   -1   2   # X C_1 C_1 X
 10      3.125000   -1   2   # X C_R C_1 X
 11      3.125000   -1   2   # X C_R C_1 C_R
 12      3.125000   -1   2   # C_R C_1 C_R X
 13      3.125000   -1   2   # X C_1 C_R X
 14      3.125000   -1   2   # X C_1 C_R X
 15      3.125000   -1   2   # X C_R C_R X
 16      3.125000   -1   2   # X C_R C_R X
 17      3.125000   -1   2   # X C_R C_1 C_R
 18      3.125000   -1   2   # X C_R C_R X
 19      3.125000   -1   2   # X C_R C_R X
"""
# Since all of them have the same parameters

VMD_dihedrals.loc[:,"dihedral type"] = 9

In [14]:
VMD_impropers["X-X-X-X"] = VMD_impropers["atom1"].map(dreiding) + "-" + \
                           VMD_impropers["atom2"].map(dreiding) + "-" + \
                           VMD_impropers["atom3"].map(dreiding) + "-" + \
                           VMD_impropers["atom4"].map(dreiding)

In [15]:
VMD_impropers["X-X-X-X"].unique()

array(['C_R-C_R-C_R-C_1', 'C_R-C_1-C_R-C_R', 'C_R-C_R-C_1-C_1',
       'C_R-C_R-C_R-C_R', 'C_1-C_R-C_R-C_1', 'C_3-C_R-C_R-C_1',
       'C_1-C_R-C_R-C_R', 'C_R-C_R-C_1-C_R', 'C_1-C_R-C_R-C_3'],
      dtype=object)

## Improper in DREIDING FF

$$
    E^d_{IJKL} = \frac{1}{2} C_I \left( \cos(\Psi) - \cos(\Psi^0_I) \right)^2
$$

For planar structures:

$$
    E_{IJKL} = K_I \left( 1 - \cos(\Psi_I) \right)
$$

With $K=40 (kcal/mol)rad^2$.

##  Improper Style in LAMMPS  `umbrella`

$$
     E = K \left( 1 - \cos(\omega) \right), \:\: \omega_0 = 0
$$

In [16]:
len(VMD_structure[VMD_structure["drediding"].isin(["C_R"])]), \
len(VMD_structure[VMD_structure["drediding"].isin(["C_1"])]), \
len(VMD_structure[VMD_structure["drediding"].isin(["C_3"])])

(2380, 310, 10)

In [17]:
VMD_structure["dreiding atom type"] = VMD_structure["drediding"].map({"C_3":10, "C_R":11, "C_1":12})

In [18]:
structure = VMD_structure[["atom id", "molecule id", "dreiding atom type", "charge", "x", "y", "z"]].copy()
structure.rename(columns={"dreiding atom type":"atom type"}, inplace=True)

In [19]:
bonds = VMD_bonds[["bond id","dreiding bond type","atom1","atom2"]].copy()
bonds.rename(columns={"dreiding bond type":"bond type"}, inplace=True)

In [20]:
angles = VMD_angles[["angle id","dreiding angle type","atom1","atom2","atom3"]].copy()
angles.rename(columns={"dreiding angle type":"angle type"}, inplace=True)

In [21]:
dihedrals = VMD_dihedrals[["dihedral id","dihedral type","atom1","atom2","atom3","atom4"]].copy()

In [22]:
impropers = VMD_impropers.copy()

In [46]:
structure["atom type"].unique()

array([12, 11, 10])

## Write Data File

In [53]:
def write_data_carbon_bed():
    f = open("carbon_bed_DREIDING_QeD_charges_.data", "w")
    f.write(f"# Created by Karina Chiñas Fuentes | Python and VMD | {datetime.today().strftime('%d.%m.%Y')}")
    f.write("\n\n")
    f.write(f"{len(structure)} atoms\n{len(bonds)} bonds\n{len(angles)} angles\n{len(dihedrals)} dihedrals\n")
    f.write(f"{len(impropers)} impropers\n13 atom types\n15 bond types\n15 angle types\n9 dihedral types\n")
    f.write("1 improper types\n\n")
    # Box size
    f.write("-19 19  xlo xhi\n")
    f.write("-19 19  ylo yhi\n")
    f.write("0   300 zlo zhi\n\n")
    # Masses
    f.write("Masses\n\n")
    f.write("1     12.011000 # CB Nafion \n")
    f.write("2     12.011000 # C Nafion \n")
    f.write("3     18.998400 # F Nafion \n")
    f.write("4     15.999400 # O Nafion \n")
    f.write("5     32.060000 # S Nafion \n")
    f.write("6     1.008     # H Water \n")
    f.write("7     15.999400 # O Water \n")
    f.write("8     1.008     # H Hydronium \n")
    f.write("9     15.999400 # O Hydronium \n")
    f.write("10    12.011000 # C C_3 Graphite \n")
    f.write("11    12.011000 # C C_R Graphite \n")
    f.write("12    12.011000 # C C_1 Graphite \n")
    f.write("13    15.999400 # O Gas \n\n")
#    f.write("14    14.0067   # N Gas \n")
#    f.write("15    1.008     # H Gas \n\n")
    f.write("Atoms\n\n")
    for i in range(len(structure)):
        f.write(f"{structure['atom id'][i]}\t{structure['molecule id'][i]}\t"+
                f"{structure['atom type'][i]}\t{structure['charge'][i]}\t"+
                f"{structure['x'][i]}\t{structure['y'][i]}\t{structure['z'][i]} # CA GRA\n")
    f.write("\nBonds\n\n")
    for i in range(len(bonds)):
        f.write(f"{bonds['bond id'][i]}\t{bonds['bond type'][i]}\t{bonds['atom1'][i]}\t{bonds['atom2'][i]}\n")
    f.write("\nAngles\n\n")
    for i in range(len(angles)):
        f.write(f"{angles['angle id'][i]}\t{angles['angle type'][i]}\t{angles['atom1'][i]}\t"+
                f"{angles['atom2'][i]}\t{angles['atom3'][i]}\n")
    f.write("\nDihedrals\n\n")
    for i in range(len(dihedrals)):
        f.write(f"{dihedrals['dihedral id'][i]}\t{dihedrals['dihedral type'][i]}\t{dihedrals['atom1'][i]}\t"+
                f"{dihedrals['atom2'][i]}\t{dihedrals['atom3'][i]}\t{dihedrals['atom4'][i]}\n")
    f.write("\nImpropers\n\n")
    for i in range(len(impropers)):
        f.write(f"{impropers['improper id'][i]}\t{impropers['improper type'][i]}\t{impropers['atom1'][i]}\t"+
                f"{impropers['atom2'][i]}\t{impropers['atom3'][i]}\t{impropers['atom4'][i]}\n")
    return

In [54]:
write_data_carbon_bed()
! code carbon_bed_DREIDING_QeD_charges_.data

In [55]:
! pwd

/home/kchinas/Documents/PhD/LAMMPS/CL/3rdMODEL/1stAttmp/data


! ls